<a href="https://colab.research.google.com/github/lakatosgabor/lama_en_edi_analyzer/blob/main/lama_en_edi_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install

In [ ]:
!sudo apt install pciutils
!pip install -q crewai
!#curl -fsSL https://ollama.com/install.sh | sh
#!pip install -qq pyngrok ollama
#!pip install langchain
#!pip install langchain_core
!pip install faiss-cpu
!pip install -q sentence-transformers
!pip install deep-translator --quiet
#!pip install langchain_community
!pip install -U langchain-ollama

In [ ]:
!nvidia-smi
import os
os.environ["OLLAMA_NUM_GPU_LAYERS"] = "100"

In [ ]:
import IPython
import subprocess
from pyngrok import ngrok
from google.colab import userdata
from typing import Any, Dict, List, Optional
import requests
import json
from google.colab import userdata
import pandas as pd
#from langchain_community.llms import Ollama
from sentence_transformers import SentenceTransformer
import numpy as np
import requests
import faiss
from langchain_ollama import OllamaLLM

from deep_translator import GoogleTranslator

A mesterséges intelligencia átalakítja a feldolgozóipart.


In [ ]:
def translate_to_hungarian(text):
    return GoogleTranslator(source='en', target='hu').translate(text)

def retrieve_context(query, k=10):
    query_vec = embed_model.encode([query])
    D, I = index.search(query_vec, k)
    return "\n".join([chunks[i] for i in I[0]])

### Server

In [ ]:
def start_ollama_server() -> None:
    """Starts the Ollama server."""
    subprocess.Popen(['ollama', 'serve'])
    print("Ollama server started.")


def check_ollama_port(port: str) -> None:
    """Check if Ollama server is running at the specified port."""
    try:
        subprocess.run(['sudo', 'lsof', '-i', '-P', '-n'], check=True, capture_output=True, text=True)
        if any(f":{port} (LISTEN)" in line for line in subprocess.run(['sudo', 'lsof', '-i', '-P', '-n'], capture_output=True, text=True).stdout.splitlines()):
            print(f"Ollama is listening on port {port}")
        else:
            print(f"Ollama does not appear to be listening on port {port}.")
    except subprocess.CalledProcessError as e:
         print(f"Error checking Ollama port: {e}")


def setup_ngrok_tunnel(port: str) -> ngrok.NgrokTunnel:
    """Sets up an ngrok tunnel.

    Args:
        port: The port to tunnel.

    Returns:
        The ngrok tunnel object.

    Raises:
        RuntimeError: If the ngrok authtoken is not set.
    """
    ngrok_auth_token = "30PNgTX5ijqAbIicSn0ZHwEOOLU_3hQRpPf6wKHW25mTj462b"
    if not ngrok_auth_token:
        raise RuntimeError("NGROK_AUTHTOKEN is not set.")

    ngrok.set_auth_token(ngrok_auth_token)
    tunnel = ngrok.connect(port, host_header=f'localhost:{port}')
    print(f"ngrok tunnel created: {tunnel.public_url}")
    return tunnel

In [ ]:
NGROK_PORT = '11434'
ngrok_tunnel = setup_ngrok_tunnel(NGROK_PORT)

ngrok tunnel created: https://aae1e437f0c7.ngrok-free.app


In [ ]:
start_ollama_server()
check_ollama_port(NGROK_PORT)

Ollama server started.
Ollama does not appear to be listening on port 11434.


In [ ]:
!ollama pull llama3

### Load LLAMA3 model

In [ ]:
llm = OllamaLLM(
    model="llama3",
    base_url="https://aae1e437f0c7.ngrok-free.app"
)

### Testing LLAMA3 model

In [ ]:
for chunk in llm.stream("What's up?"):
    print(chunk, end="", flush=True)

Not much! Just an AI hanging out, waiting to chat with you. How about you?

### Load and chunk dataset

In [ ]:
df = pd.read_csv("/content/edi.csv")
print(df.head(100))

In [ ]:
with open("/content/edi.csv", "r", encoding="utf-8") as f:
    text = f.read()

chunk_size = 500
chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
print(f"✅ {len(chunks)} chunk készült")

✅ 7 chunk készült


### Create embeddings

In [ ]:
embed_model = SentenceTransformer("all-MiniLM-L6-v2")  # all-mpnet-base-v2
embeddings = embed_model.encode(chunks, convert_to_numpy=True)
print("✅ Embeddingek kész")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


✅ Embeddingek kész


In [ ]:
d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(embeddings)
print("✅ FAISS index létrehozva")

✅ FAISS index létrehozva


### Run the model, and  Analyze the data




In [ ]:
query = "List the orders where the customer modified the order quantity within 3 days."

#query = "List orders where the customer modified the order quantity within 3 days. Only include quantities that are divisible by 3."


context = retrieve_context(query)

prompt = f"""You are a customer contact. You are working on a dataset containing customer orders: {context}
              Analyze the data set based on the question, then answer briefly and clearly. Consider the entire data set!
              Question: {query}
          """

answer_full_text=''
for chunk in llm.stream(prompt):
    print(chunk, end="", flush=True)
    answer_full_text+=chunk

In [ ]:
hungarian_text = translate_to_hungarian(answer_full_text)
print(hungarian_text)